In [1]:
import os 
import re
import json 
import pandas as pd

from datetime import datetime

In [21]:
data = []

In [22]:
dir = "data/music_related/"
ignore_files = ["all_music_tweets.ndjson", ".ipynb_checkpoints"] # ignore these files in the iteration 

for file in os.listdir(dir): #read all the files in the directory
    if file not in ignore_files: #ignore the two files from above
        with open(dir + file) as f: #open each file
            lines = f.readlines() #read all the lines in the file
            
            for line in lines: #looping through the tweets in each file
                tweet = json.loads(line.strip()) #tweet now contains the atributes of the line (tweet) we're looping through 
                
                if not tweet["text"].startswith("RT @"): #excluding RT's
                    tweet_data = {} #saving the coloumns we want from non-RT tweets
                    tweet_data["id"] = tweet["id"] #tweet ID
                    tweet_data["user_id"] = tweet["from_user_id"] #user ID
                         
                    date = datetime.strptime(tweet["created_at"], "%a %b %d %H:%M:%S %z %Y") #re-formatting date
                    tweet_data["date"] = date.date() #date
                    
                    url = re.findall("https:\/\/t.co\/[a-zA-Z0-9]{10}", tweet["text"])
                    if len(url) > 0:
                        tweet_data["link"] = url #link from tweets
                    else : 
                        tweet_data["link"] = ""

                    tweet_data["texts"] = re.sub("https:\/\/t.co\/[a-zA-Z0-9]{10}", "", tweet["text"]) #text + removing the link
                    
                    tweet_data["country"] = tweet["user_location"]["country_code"] #country
                    if "state" in tweet["user_location"].keys(): #some tweets don't have states, but if they do we want it
                        tweet_data["state"] = tweet["user_location"]["state"] #state
                    else: 
                        tweet_data["state"] = "" #otherwise they are empty 
               
                    data.append(tweet_data) #putting it all together

In [23]:
df = pd.DataFrame(data=data) #making it a dataframe

In [24]:
df.to_csv("test.csv", index=False) #saving as a csv-file